In [205]:
import numpy as np
import math
import random

در این تابع ابتدا داده را ورودی میگیریم

In [206]:
def open_file(i):
    # Open input file
    if (i == 1):
        infile = open('input1.stock', 'r')
    elif (i == 2):
        infile = open('input2.stock', 'r')
    elif (i == 3):
        infile = open('input3.stock', 'r')
    else:
        infile = open('input4.stock', 'r')

    # Read instance header for the pr1002
    length = int(infile.readline().strip().split()[2]) #length
    infile.readline()
    infile.readline()
    requests = np.array(list(map(int, infile.readline().strip().split(', ')[:-1]))) #requests

    # Close input file
    infile.close()
    
    return length, requests

این تابع محاسبه هزینه است. به این صورت عمل می‌کند که به صورت متوالی تا زمانی جمع تکه ها بزرگ‌تر از تکه اصلی نشده، جزو بک تکه حساب می‌کند و تعداد تکه‌های مورد نیاز را محاسبه می‌کند که باید آن را مینیمم کنیم.

In [207]:
def calculate_cost(orders, length):
    cost = 0
    s = 0
    i = 0
    while(s <= length and i < len(orders)):
        s += orders[i]
        if (s > length):
            s -= orders[i]
            i -= 1
            cost += 1
            s = 0
        i += 1
    return cost

این تابع محاسبه همسایگی است. به این صورت عمل می‌کند که دو تکه را به صورت تصادفی با هم تعویض می‌کند، اگر هزینه کم‌تر شد، آن را جابه‌جا می‌کند. اگر هم بهتر نبود، با یک شانس که بسته به دما و اختلاف هزینه‌ها است، تعویض می کند.

In [208]:
def neighbor(orders, temperature, length):
    new_orders = orders.copy()
    i = random.randint(0, len(orders) - 1)
    j = random.randint(0, len(orders) - 1)
    
    temp = new_orders[i]
    new_orders[i] = new_orders[j]
    new_orders[j] = temp
    
    cost = calculate_cost(orders, length)
    new_cost = calculate_cost(new_orders, length)
    
    if (new_cost < cost):
        orders = new_orders
        return orders
        
    rand = random.random()
    if (rand < math.exp((cost - new_cost)/temperature)):
        orders = new_orders
    
    return orders

این تابع اصلی است. به این صورت عمل کرده که ابتدا یک جایگشت می‌سازیم. سپس همسایه را محاسبه می‌کنیم و سپس بهترین را به‌روز می‌کنیم و بعد دما را کم می‌کنیم.

In [209]:
def simulated_annealing(temperature, alpha, TL, i):
    length, requests = open_file(i)
    
    requests = np.random.permutation(requests)

    best_order = []
    least_cost = np.Inf
    costs = []
    best_costs = []
    while(temperature > 0.1):
        for i in range(TL):
            requests = neighbor(requests, temperature, length)
            
            if (calculate_cost(requests, length) < least_cost):
                best_order = requests
                least_cost = calculate_cost(requests, length)
                
            costs.append(calculate_cost(requests, length))
            best_costs.append(least_cost)
        temperature *= alpha
        
    return best_order, least_cost, costs, best_costs

نتیجه تست اول که 51 است.

In [210]:
order, least, costs, best_costs = simulated_annealing(1, 0.99, 1000, 1) #input = input1.stock
print(order)
print(least)

[653 555 232  18 107 118 788  69 933 660 187  71 284 678 251 618 283 170
 544 517 109 337  23 125  92 506 224 805 312 180 368 354  88 278 268 672
 301 292 515 181 495 149 266 371 627 532 437 218 126 457 365 441  46 753
  80 145 186 648 788 144 609  43  78 868 914  75 581 246 148 412  92 405
 592 264 106 106 116 686 346 501 119 753  45 117  60 967 549 312 409 557
 463 518  33 333  99 525 460 507 987 241 689  84 230 286  70 266 123 106
 716 149 414 106 286 249 115 557  61 171 295 125 370 351 557 306 211 280
  88 402 135 424  53 356 315 988 662 248  79 149 632]
51


نتیجه تست دوم که 78 است.

In [211]:
order, least, costs, best_costs = simulated_annealing(1, 0.99, 1000, 2) #input = input2.stock
print(order)
print(least)

[2100 2200 1380 2100 2050 2200 2200 2150 1380 1710 1820 1820 1380 1820
 1820 1880 2140 1380 2000 1820 1820 1710 1710 1380 2050 1520 2200 1520
 1880 1710 1710 1820 1560 2140 1380 2200 1820 2140 2000 1380 2050 1560
 2100 2200 2000 2050 1520 1710 1820 1520 2140 2150 1380 1820 2050 1930
 1520 2140 2200 2140 1930 1560 1560 1880 1380 1930 2100 1930 1520 1710
 1930 1820 1560 1880 2100 1560 2100 1880 1820 1880 1930 1520 2140 1380
 2200 1710 2150 2150 2000 2050 1520 2150 2200 1560 2150 1880 2140 1380
 1880 1520 2200 1560 2100 1820 1560 2200 1930 1710 2050 1380 2150 1520
 1520 2150 1930 1520 2050 1520 2000 1520 2150 1880 1930 2200 1880 2200
 1380 1930 1520 2100 1380 2100 2100 1380 1930 2150 2050 1380 2100 1560
 1820 2140 1380 2150 1820 2000 2000 1520 2000 2150 1380 1880 1560 1930
 2150 2200 2100 1710 1520 1820 2050 1710 1880 1520 2140 1880 2200 1520
 2050 1930 1520 2200 2000 2140 1380 1930 1380 1930 2140 1710 1880 1520
 2140 1820 1930 2150 1520 1560 2100 1880 1820 2000 2150 2200 1710 1880
 1880 

نتیجه تست سوم که 96 است.

In [214]:
order, least, costs, best_costs = simulated_annealing(1, 0.99, 1000, 3) #input = input3.stock
print(order)
print(least)

[  7   8  52  99 255  49   2   5  10  11  29 403   2   3   5 161   1   1
 313   5 421 405   4 280   3  14   6   3  10   8 174   8 229   3  91   5
   1  18   7  21  87  85   4   8   1   4   6 359   6   4 152   1   7  12
 277   3  16  16  50 159 275  18   1  22 144   4  23 264   2  13   9 251
   2   7   2 225  18   6 116   2  14   4  13   1 167  37  90   1 147  41
   1 134   2 169   5 110  12  17   1 152 134  11  10   9   2   6  11   2
 335   1 152   1 172  13   1   9 170   6   2 102   5   8  11   6   2   2
  19   3  92   3   7   9   2  10   5   3 321  14   1   8 108 318   3   4
  10   2  10   5   2   8  21   4   1   3  43   3 153   1  18 225   8   1
   5   1   5   9 271 133  16   3  26  88   2   6 245   6   1   2  17   7
   6  14   9  27   4   7  10   4 204 239  26 196  16   1 279   1   2   1
   3   4   9  45  40 250   5   7   7   4  10  93  78  12   9  14   4   8
   8   3   4 152   2 199   3 263   1   7  16 162   2   7   3   8   9 386
   1   1   1  96   3   9  11   9   2   2   9   7 17

نتیجه تست چهارم که 218 است.

In [213]:
order, least, costs, best_costs = simulated_annealing(1, 0.99, 1000, 4) #input = input4.stock
print(order)
print(least)

[ 1 45  6 27  5  6 36 37 22 80 10  9 32 16 52 22 29  4 30 12  9 43  6 10
  2  6 21 17 58  6 13 31 51 15  3 32 46 20 13  3  9 73 37 11 20 28  2  4
 24 68 54 17  8  6  1 80 18 41 19 29  4  4 10  5 16  9 44 23  1 23  2 20
  5 20 44 26 11 15 10 38  6 29 14 59  6 20  8  7 73 19 46 54  7  8 16 67
 20 16  5  2 40  4 20 46  1 25 34  3 32 30  9 53  4 30 30 24 25 28 12  5
  2 28 24 21 36  4 17 18 63 35 98 76 10 15 68 15  2 79  7  7 13 13  4 10
 52  5 33  1 10  9 45 39 33  8 12  3 36 18 22 23  7  5 65  4 18 25  8  3
  3  1  8 47 65  1 11 56  2  1 39  2 37 13 14  8 18  8 20  2 47 25 16 24
  6  3  7 38 65 33  5 20  7 14 38  7 15  9 23 44  7 11 14  5  9  9 21 16
 87  1 36 50  3  5  3 12 81 32 41  7  2 21 78  4  2 32  1 30  4  4 14 18
 16  4  6  4  5 36  5  3  1 60  8  5 11  7 21 59  6 67 27 48 34 18 11 68
 13  8  5 27  3 47 11  6 17 43  6  9 22  1 20 13  9 24 34 26 13 33 16 51
 12  2  5  3 14 18 10  4 15  7 22  6 15  3  3  5 16  4 15  4 24 26 20 65
 11 73 23  3  5  4 17  3 15 51 21 13 17  5  3  2 16

سرعت کم شدن دما 0.01 است و توی هر دما، ۱۰۰۰ بار اجرا می‌شود
که نتیجه مطلوبی داشت.

عملکرد دما در شانس تغییر ترتیب مهم است.
ممکن است باعث شود از بهینه کل فاصله بگیریم و یا اینکه در بهینه محلی گیر کنیم.

از تابع خطی می‌توان به شرط نزولی بودن استفاده کرد ولی ممکن است از بهینه سراسری خارج شویم(به آسانی).
در واقع چون نرخ کم شدن دما، خیلی کم است.

In [216]:
order, least, costs, best_costs = simulated_annealing(100, 0.99, 10, 1) #input = input1.stock
print(order)
print(least)

[218 662 106 264 678 557 106 424 251 753 788 405 592 187 117 557  60 286
 266 119 301 441 368 632 333 437 501 460 414 109 967 627 149 241 518  33
 145 987 555 506 371  23 753 232 557 115 409 116  75 149 402  71  78  61
 933  88 230 495 144 118 354 356 171 148  92 716 266 412 284  92 123 283
 135 988 805 868 107 672  70 211 246 463 315 457 914 686  99 660 224 581
 365 609 351 125 618 532 370  43 648 126  84 653  46 181  69 186 788  79
  88  18  80 292 346  53 268 689 106 286 180 306 517 249 515 337 149 170
 549 248 125 507 525 280  45 312 106 295 278 544 312]
54


In [217]:
order, least, costs, best_costs = simulated_annealing(0.5, 0.99, 10, 1) #input = input1.stock
print(order)
print(least)

[148 181 653 660 356 557 678 312  60 501 268 788 592  53 224 518 241  75
  43 119 672 292 689 144 187 753 368 555  46 402 337 370  79 125 405 618
 301 286 648 264 441 412 507 180 662 171 371 354 106 117  88 284 283  92
 351 218 232 306 581 286  80 532 409 123  99 149 557 424 315 249 266 987
 515 280 686 109 106 506 414 716 251 116 278 437 632  33  84  69  61 967
 346 135 295 106  71 868 988 517 457  23 312  78 557 463 460  18 788 125
 126 149 627 149 609 115 106  70  92 753  88 211 549  45 145 118 333 365
 107 544 495 246 186 248 266 230 525 933 805 170 914]
53


می‌بینیم که با افزایش یا کاهش دما، تفاوت آنچنانی در این مسئله به وجود نمی‌آید.